<a href="https://colab.research.google.com/github/ENGRMike/genre-prediction/blob/main/genre_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/ML/Data/imdb_genres/

cleaned_movie_data.csv	       metadata.csv	     movie_data_one_hot.csv
data-20210910T014654Z-001.zip  movie_data_embed.csv  plot_summaries.txt
Genre-Notes.csv		       movie_data_lemma.csv


In [ ]:
data_location = "/content/gdrive/MyDrive/ML/Data/imdb_genres/"
metadata_csv = "metadata.csv"
plot_csv = "plot_summaries.txt"

In [ ]:
import pandas as pd

In [ ]:
metadata_df = pd.read_csv(f"{data_location}{metadata_csv}", index_col=False, converters={"languages": eval, "countries": eval, "genres": eval})
plots_df = pd.read_csv(f"{data_location}{plot_csv}", sep="\t", header=None)
plots_df.columns = ["movie_id", "plot"]

In [ ]:
metadata_df.head(5)

,movie_id,movie_name,release_date,revenue,runtime,languages,countries,genres
0,975900,ghosts of mars,2001-08-24,14010832.0,98.0,[english language],[united states of america],"[action, horror/thriller, sci-fi/fantasy, west..."
1,9363483,white of the eye,1987,NaN,110.0,[english language],[united kingdom],[horror/thriller]
2,261236,a woman in flames,1983,NaN,106.0,[german language],[germany],[drama]
3,18998739,the sorcerer's apprentice,2002,NaN,86.0,[english language],[south africa],"[action, drama, sci-fi/fantasy]"
4,6631279,little city,1997-04-04,NaN,93.0,[english language],[united states of america],"[comedy, drama, romance]"


In [ ]:
plots_df.head(5)

,movie_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [ ]:
movies_df = metadata_df.merge(plots_df, on="movie_id", how="inner")

In [ ]:
movies_df.head(5)

,movie_id,movie_name,release_date,revenue,runtime,languages,countries,genres,plot
0,975900,ghosts of mars,2001-08-24,14010832.0,98.0,[english language],[united states of america],"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th..."
1,9363483,white of the eye,1987,NaN,110.0,[english language],[united kingdom],[horror/thriller],A series of murders of rich young women throug...
2,261236,a woman in flames,1983,NaN,106.0,[german language],[germany],[drama],"Eva, an upper class housewife, becomes frustra..."
3,18998739,the sorcerer's apprentice,2002,NaN,86.0,[english language],[south africa],"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns..."
4,6631279,little city,1997-04-04,NaN,93.0,[english language],[united states of america],"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a..."


In [ ]:
movie_clean_df = movies_df[["movie_name", "genres", "plot"]].copy()

In [ ]:
import re

def clean_text(text):
  text = re.sub("\'", "", text)
  text = re.sub("[^a-zA-Z]", " ", text)
  text = " ".join(text.split())
  text = text.lower()

  return text

In [ ]:
movie_clean_df["clean_plot"] = movie_clean_df["plot"].apply(lambda x: clean_text(x))

movie_clean_df.head(5)

,movie_name,genres,plot,clean_plot
0,ghosts of mars,"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th...",set in the second half of the nd century the f...
1,white of the eye,[horror/thriller],A series of murders of rich young women throug...,a series of murders of rich young women throug...
2,a woman in flames,[drama],"Eva, an upper class housewife, becomes frustra...",eva an upper class housewife becomes frustrate...
3,the sorcerer's apprentice,"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns...",every hundred years the evil morgana returns t...
4,little city,"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a...",adam a san francisco based artist who works as...


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

def remove_stopwords(text):
  text = [w for w in text.split() if not w in stopwords]
  return " ".join(text)

In [ ]:
movie_clean_df["clean_plot"] = movie_clean_df["clean_plot"].apply(lambda x: remove_stopwords(x))

In [ ]:
movie_clean_df.head(5)

,movie_name,genres,plot,clean_plot
0,ghosts of mars,"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th...",set second half nd century film depicts mars p...
1,white of the eye,[horror/thriller],A series of murders of rich young women throug...,series murders rich young women throughout ari...
2,a woman in flames,[drama],"Eva, an upper class housewife, becomes frustra...",eva upper class housewife becomes frustrated l...
3,the sorcerer's apprentice,"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns...",every hundred years evil morgana returns claim...
4,little city,"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a...",adam san francisco based artist works cab driv...


In [ ]:
nltk.download("wordnet")
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
  return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
%time movie_clean_df["lemma_plot"] = movie_clean_df["clean_plot"].apply(lemmatize_text)

CPU times: user 36.3 s, sys: 1.08 s, total: 37.4 s
Wall time: 37.5 s


In [ ]:
movie_clean_df.head()

,movie_name,genres,plot,clean_plot,lemma_plot
0,ghosts of mars,"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th...",set second half nd century film depicts mars p...,"[set, second, half, nd, century, film, depicts..."
1,white of the eye,[horror/thriller],A series of murders of rich young women throug...,series murders rich young women throughout ari...,"[series, murder, rich, young, woman, throughou..."
2,a woman in flames,[drama],"Eva, an upper class housewife, becomes frustra...",eva upper class housewife becomes frustrated l...,"[eva, upper, class, housewife, becomes, frustr..."
3,the sorcerer's apprentice,"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns...",every hundred years evil morgana returns claim...,"[every, hundred, year, evil, morgana, return, ..."
4,little city,"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a...",adam san francisco based artist works cab driv...,"[adam, san, francisco, based, artist, work, ca..."


In [ ]:
movie_clean_df.to_csv(f"{data_location}movie_data_lemma.csv")

In [ ]:
movie_df = pd.read_csv(f"{data_location}movie_data_lemma.csv", converters={"genres": eval, "lemma_plot": eval})
movie_df.head()

,Unnamed: 0,movie_name,genres,plot,clean_plot,lemma_plot
0,0,ghosts of mars,"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th...",set second half nd century film depicts mars p...,"[set, second, half, nd, century, film, depicts..."
1,1,white of the eye,[horror/thriller],A series of murders of rich young women throug...,series murders rich young women throughout ari...,"[series, murder, rich, young, woman, throughou..."
2,2,a woman in flames,[drama],"Eva, an upper class housewife, becomes frustra...",eva upper class housewife becomes frustrated l...,"[eva, upper, class, housewife, becomes, frustr..."
3,3,the sorcerer's apprentice,"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns...",every hundred years evil morgana returns claim...,"[every, hundred, year, evil, morgana, return, ..."
4,4,little city,"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a...",adam san francisco based artist works cab driv...,"[adam, san, francisco, based, artist, work, ca..."


In [ ]:
one_hot = pd.get_dummies(movie_df["genres"].apply(pd.Series).stack()).sum(level=0)
one_hot.head()

,Misc,action,comedy,documentary,drama,horror/thriller,romance,sci-fi/fantasy,western
0,0,1,0,0,0,1,0,1,1
1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,1,0,0,1,0,0,1,0
4,0,0,1,0,1,0,1,0,0


In [ ]:
df = movie_df.join(one_hot)
df.head(5)

,Unnamed: 0,movie_name,genres,plot,clean_plot,lemma_plot,Misc,action,comedy,documentary,drama,horror/thriller,romance,sci-fi/fantasy,western
0,0,ghosts of mars,"[action, horror/thriller, sci-fi/fantasy, west...","Set in the second half of the 22nd century, th...",set second half nd century film depicts mars p...,"[set, second, half, nd, century, film, depicts...",0,1,0,0,0,1,0,1,1
1,1,white of the eye,[horror/thriller],A series of murders of rich young women throug...,series murders rich young women throughout ari...,"[series, murder, rich, young, woman, throughou...",0,0,0,0,0,1,0,0,0
2,2,a woman in flames,[drama],"Eva, an upper class housewife, becomes frustra...",eva upper class housewife becomes frustrated l...,"[eva, upper, class, housewife, becomes, frustr...",0,0,0,0,1,0,0,0,0
3,3,the sorcerer's apprentice,"[action, drama, sci-fi/fantasy]","Every hundred years, the evil Morgana returns...",every hundred years evil morgana returns claim...,"[every, hundred, year, evil, morgana, return, ...",0,1,0,0,1,0,0,1,0
4,4,little city,"[comedy, drama, romance]","Adam, a San Francisco-based artist who works a...",adam san francisco based artist works cab driv...,"[adam, san, francisco, based, artist, work, ca...",0,0,1,0,1,0,1,0,0


In [ ]:
df.to_parquet(f"{data_location}movie_data_one_hot.parquet")